##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/federated/tutorials/tff_for_federated_learning_research_compression"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/federated/blob/v0.36.0/docs/tutorials/tff_for_federated_learning_research_compression.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/federated/blob/v0.36.0/docs/tutorials/tff_for_federated_learning_research_compression.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/federated/docs/tutorials/tff_for_federated_learning_research_compression.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

# 페더레이션 학습 연구를 위한 TFF: 압축 모델링 및 업데이트

**참고**: 이 colab은 <code>tensorflow_federated</code> pip 패키지의 <a>최신 릴리즈 버전</a>에서 동작하는 것으로 확인되었지만, Tensorflow Federated 프로젝트는 아직 릴리즈 전 개발 중이며 `master`에서 동작하지 않을 수 있습니다.

In this tutorial, we use the [EMNIST](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/emnist) dataset to demonstrate how to enable lossy compression algorithms to reduce communication cost in the Federated Averaging algorithm using the `tff.learning` API. For more details on the Federated Averaging algorithm, see the paper [Communication-Efficient Learning of Deep Networks from Decentralized Data](https://arxiv.org/abs/1602.05629).

## 시작하기 전에

시작하기 전에 다음을 실행하여 환경이 올바르게 설정되었는지 확인합니다. 인사말이 표시되지 않으면 [설치](../install.md) 가이드에서 지침을 참조하세요.

In [ ]:
#@test {"skip": true}
!pip install --quiet --upgrade tensorflow-federated
!pip install --quiet --upgrade tensorflow-model-optimization
!pip install --quiet --upgrade nest-asyncio

import nest_asyncio
nest_asyncio.apply()

In [1]:
%load_ext tensorboard

import functools

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

TFF가 동작하는지 확인합니다.

In [2]:
@tff.federated_computation
def hello_world():
  return 'Hello, World!'

hello_world()

b'Hello, World!'

## 입력 데이터 준비하기

이 섹션에서는 TFF에 포함된 EMNIST 데이터세트를 로드하고 전처리합니다. EMNIST 데이터세트에 대한 자세한 내용은 [이미지 분류를 위한 Federated Learning](https://www.tensorflow.org/federated/tutorials/federated_learning_for_image_classification#preparing_the_input_data) 튜토리얼을 확인하세요.


In [3]:
# This value only applies to EMNIST dataset, consider choosing appropriate
# values if switching to other datasets.
MAX_CLIENT_DATASET_SIZE = 418

CLIENT_EPOCHS_PER_ROUND = 1
CLIENT_BATCH_SIZE = 20
TEST_BATCH_SIZE = 500

emnist_train, emnist_test = tff.simulation.datasets.emnist.load_data(
    only_digits=True)

def reshape_emnist_element(element):
  return (tf.expand_dims(element['pixels'], axis=-1), element['label'])

def preprocess_train_dataset(dataset):
  """Preprocessing function for the EMNIST training dataset."""
  return (dataset
          # Shuffle according to the largest client dataset
          .shuffle(buffer_size=MAX_CLIENT_DATASET_SIZE)
          # Repeat to do multiple local epochs
          .repeat(CLIENT_EPOCHS_PER_ROUND)
          # Batch to a fixed client batch size
          .batch(CLIENT_BATCH_SIZE, drop_remainder=False)
          # Preprocessing step
          .map(reshape_emnist_element))

emnist_train = emnist_train.preprocess(preprocess_train_dataset)

## 모델 정의하기

여기에서는 원래 FedAvg CNN을 기반으로 keras 모델을 정의한 다음 TFF에서 사용할 수 있도록 [tff.learning.Model](https://www.tensorflow.org/federated/api_docs/python/tff/learning/Model) 인스턴스에 keras 모델을 래핑합니다.

단순히 모델을 직접 생성하는 대신 모델을 생성하는 **함수**가 필요합니다. 또한, 이 함수는 미리 구성된 모델을 캡처할 수 **없습니다**. 함수가 호출된 컨텍스트에서 모델을 만들어야 합니다. 그 이유는 TFF는 기기로 이동하도록 설계되었으며 리소스를 캡처하고 패키징할 수 있도록 리소스의 구성 시기를 제어해야 하기 때문입니다.

In [4]:
def create_original_fedavg_cnn_model(only_digits=True):
  """The CNN model used in https://arxiv.org/abs/1602.05629."""
  data_format = 'channels_last'

  max_pool = functools.partial(
      tf.keras.layers.MaxPooling2D,
      pool_size=(2, 2),
      padding='same',
      data_format=data_format)
  conv2d = functools.partial(
      tf.keras.layers.Conv2D,
      kernel_size=5,
      padding='same',
      data_format=data_format,
      activation=tf.nn.relu)

  model = tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(28, 28, 1)),
      conv2d(filters=32),
      max_pool(),
      conv2d(filters=64),
      max_pool(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(512, activation=tf.nn.relu),
      tf.keras.layers.Dense(10 if only_digits else 62),
      tf.keras.layers.Softmax(),
  ])

  return model

# Gets the type information of the input data. TFF is a strongly typed
# functional programming framework, and needs type information about inputs to 
# the model.
input_spec = emnist_train.create_tf_dataset_for_client(
    emnist_train.client_ids[0]).element_spec

def tff_model_fn():
  keras_model = create_original_fedavg_cnn_model()
  return tff.learning.from_keras_model(
      keras_model=keras_model,
      input_spec=input_spec,
      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

## 모델 훈련 및 훈련 메트릭 출력

이제 Federated Averaging 알고리즘을 구성하고 EMNIST 데이터세트에 대해 정의된 모델을 훈련할 준비가 되었습니다.

First we need to build a Federated Averaging algorithm using the [tff.learning.algorithms.build_weighted_fed_avg](https://www.tensorflow.org/federated/api_docs/python/tff/learning/algorithms/build_weighted_fed_avg) API.

In [5]:
federated_averaging = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn=tff_model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0))

이제 Federated Averaging 알고리즘을 실행해 보겠습니다. TFF의 관점에서 Federated Learning 알고리즘의 실행은 다음과 같습니다.

1. 알고리즘을 초기화하고 초기 서버 상태를 가져옵니다. 서버 상태에는 알고리즘을 수행하는 데 필요한 정보가 포함되어 있습니다. TFF는 함수형이므로 이 상태에는 알고리즘이 사용하는 모든 최적화 상태(예: momentum 조건)와 모델 매개변수 자체가 모두 포함됩니다. 이 상태는 인수로 전달되고 TFF 계산의 결과로 반환됩니다.
2. 알고리즘을 라운드별로 실행합니다. 각 라운드에서 각 클라이언트가 데이터에 대해 모델을 훈련한 결과로서 새로운 서버 상태가 반환됩니다. 일반적으로 한 라운드에서 다음과 같습니다.
    1. 서버는 모든 참여 클라이언트로 모델을 브로드캐스팅합니다.
    2. 각 클라이언트는 모델과 자체 데이터를 기반으로 작업을 수행합니다.
    3. 서버는 모든 모델을 집계하여 새 모델을 포함하는 서버 상태를 생성합니다.

자세한 내용은 [사용자 정의 Federated Algorithm, 2부: Federated Averaging 구현하기](https://www.tensorflow.org/federated/tutorials/custom_federated_algorithms_2) 튜토리얼을 참조하세요.

훈련 메트릭은 훈련 후 표시하기 위해 Tensorboard 디렉토리에 기록됩니다.

In [6]:
#@title Load utility functions

def format_size(size):
  """A helper function for creating a human-readable size."""
  size = float(size)
  for unit in ['bit','Kibit','Mibit','Gibit']:
    if size < 1024.0:
      return "{size:3.2f}{unit}".format(size=size, unit=unit)
    size /= 1024.0
  return "{size:.2f}{unit}".format(size=size, unit='TiB')

def set_sizing_environment():
  """Creates an environment that contains sizing information."""
  # Creates a sizing executor factory to output communication cost
  # after the training finishes. Note that sizing executor only provides an
  # estimate (not exact) of communication cost, and doesn't capture cases like
  # compression of over-the-wire representations. However, it's perfect for
  # demonstrating the effect of compression in this tutorial.
  sizing_factory = tff.framework.sizing_executor_factory()

  # TFF has a modular runtime you can configure yourself for various
  # environments and purposes, and this example just shows how to configure one
  # part of it to report the size of things.
  context = tff.framework.ExecutionContext(executor_fn=sizing_factory)
  tff.framework.set_default_context(context)

  return sizing_factory

In [7]:
def train(federated_averaging_process, num_rounds, num_clients_per_round, summary_writer):
  """Trains the federated averaging process and output metrics."""
  # Create a environment to get communication cost.
  environment = set_sizing_environment()

  # Initialize the Federated Averaging algorithm to get the initial server state.
  state = federated_averaging_process.initialize()

  with summary_writer.as_default():
    for round_num in range(num_rounds):
      # Sample the clients parcitipated in this round.
      sampled_clients = np.random.choice(
          emnist_train.client_ids,
          size=num_clients_per_round,
          replace=False)
      # Create a list of `tf.Dataset` instances from the data of sampled clients.
      sampled_train_data = [
          emnist_train.create_tf_dataset_for_client(client)
          for client in sampled_clients
      ]
      # Round one round of the algorithm based on the server state and client data
      # and output the new state and metrics.
      result = federated_averaging_process.next(state, sampled_train_data)
      state = result.state
      train_metrics = result.metrics['client_work']['train']

      # For more about size_info, please see https://www.tensorflow.org/federated/api_docs/python/tff/framework/SizeInfo
      size_info = environment.get_size_info()
      broadcasted_bits = size_info.broadcast_bits[-1]
      aggregated_bits = size_info.aggregate_bits[-1]

      print('round {:2d}, train_metrics={}, broadcasted_bits={}, aggregated_bits={}'.format(
          round_num, train_metrics, format_size(broadcasted_bits), format_size(aggregated_bits)))

      # Add metrics to Tensorboard.
      for name, value in train_metrics.items():
          tf.summary.scalar(name, value, step=round_num)

      # Add broadcasted and aggregated data size to Tensorboard.
      tf.summary.scalar('cumulative_broadcasted_bits', broadcasted_bits, step=round_num)
      tf.summary.scalar('cumulative_aggregated_bits', aggregated_bits, step=round_num)
      summary_writer.flush()

In [8]:
# Clean the log directory to avoid conflicts.
try:
  tf.io.gfile.rmtree('/tmp/logs/scalars')
except tf.errors.OpError as e:
  pass  # Path doesn't exist

# Set up the log directory and writer for Tensorboard.
logdir = "/tmp/logs/scalars/original/"
summary_writer = tf.summary.create_file_writer(logdir)

train(federated_averaging_process=federated_averaging, num_rounds=10,
      num_clients_per_round=10, summary_writer=summary_writer)

round  0, train_metrics=OrderedDict([('sparse_categorical_accuracy', 0.092454836), ('loss', 2.310193), ('num_examples', 941), ('num_batches', 51)]), broadcasted_bits=507.62Mibit, aggregated_bits=507.62Mibit
round  1, train_metrics=OrderedDict([('sparse_categorical_accuracy', 0.10029791), ('loss', 2.3102622), ('num_examples', 1007), ('num_batches', 55)]), broadcasted_bits=1015.24Mibit, aggregated_bits=1015.25Mibit
round  2, train_metrics=OrderedDict([('sparse_categorical_accuracy', 0.10710711), ('loss', 2.3048222), ('num_examples', 999), ('num_batches', 54)]), broadcasted_bits=1.49Gibit, aggregated_bits=1.49Gibit
round  3, train_metrics=OrderedDict([('sparse_categorical_accuracy', 0.1061061), ('loss', 2.3066027), ('num_examples', 999), ('num_batches', 55)]), broadcasted_bits=1.98Gibit, aggregated_bits=1.98Gibit
round  4, train_metrics=OrderedDict([('sparse_categorical_accuracy', 0.1287594), ('loss', 2.2999024), ('num_examples', 1064), ('num_batches', 58)]), broadcasted_bits=2.48Gibit, a

위에서 지정한 루트 로그 디렉터리로 TensorBoard를 시작하여 훈련 메트릭을 표시합니다. 데이터를 로드하는 데 몇 초 정도 걸릴 수 있습니다. 손실 및 정확성을 제외하고는 브로트캐스팅 및 집계된 데이터의 양도 출력합니다. 브로드캐스팅된 데이터는 서버가 각 클라이언트에 푸시하는 텐서를 의미하며, 집계 데이터는 각 클라이언트가 서버에 반환하는 텐서를 의미합니다.

In [ ]:
#@test {"skip": true}
%tensorboard --logdir /tmp/logs/scalars/ --port=0

## Build a custom aggregation function

Now let's implement function to use lossy compression algorithms on aggregated data. We will use TFF's API to create a `tff.aggregators.AggregationFactory` for this. While researchers may often want to implement their own (which can be done via the `tff.aggregators` API), we will use a built-in method for doing so, specifically `tff.learning.compression_aggregator`.

It is important to note that this aggregator does not apply compression to the entire model at once. Instead, applies compression to only those variables in the model that are sufficiently large. Generally, small variables such as biases are more sensitive to inaccuracy, and being relatively small, the potential communication savings are also relatively small.

In [12]:
compression_aggregator = tff.learning.compression_aggregator()
isinstance(compression_aggregator, tff.aggregators.WeightedAggregationFactory)

True

Above, you can see that the compression aggregator is a *weighted* aggregation factory, which means that it involves weighted aggregation (in contrast to aggregators meant for differential privacy, which are often unweighted).

This aggregation factory can be direclty plugged into FedAvg via its `model_aggregator` argument.

In [13]:
federated_averaging_with_compression = tff.learning.algorithms.build_weighted_fed_avg(
    tff_model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02),
    server_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=1.0),
    model_aggregator=compression_aggregator)

## 모델 다시 훈련하기

이제 새로운 Federated Averaging 알고리즘을 실행해 보겠습니다.

In [14]:
logdir_for_compression = "/tmp/logs/scalars/compression/"
summary_writer_for_compression = tf.summary.create_file_writer(
    logdir_for_compression)

train(federated_averaging_process=federated_averaging_with_compression, 
      num_rounds=10,
      num_clients_per_round=10,
      summary_writer=summary_writer_for_compression)

round  0, train_metrics=OrderedDict([('sparse_categorical_accuracy', 0.087804876), ('loss', 2.3126457), ('num_examples', 1025), ('num_batches', 55)]), broadcasted_bits=507.62Mibit, aggregated_bits=146.47Mibit
round  1, train_metrics=OrderedDict([('sparse_categorical_accuracy', 0.073267326), ('loss', 2.3111901), ('num_examples', 1010), ('num_batches', 56)]), broadcasted_bits=1015.24Mibit, aggregated_bits=292.93Mibit
round  2, train_metrics=OrderedDict([('sparse_categorical_accuracy', 0.08925144), ('loss', 2.3071017), ('num_examples', 1042), ('num_batches', 57)]), broadcasted_bits=1.49Gibit, aggregated_bits=439.40Mibit
round  3, train_metrics=OrderedDict([('sparse_categorical_accuracy', 0.07985144), ('loss', 2.3061485), ('num_examples', 1077), ('num_batches', 59)]), broadcasted_bits=1.98Gibit, aggregated_bits=585.86Mibit
round  4, train_metrics=OrderedDict([('sparse_categorical_accuracy', 0.11947791), ('loss', 2.302166), ('num_examples', 996), ('num_batches', 55)]), broadcasted_bits=2.48

TensorBoard를 다시 시작하여 두 실행 간의 훈련 메트릭을 비교합니다.

As you can see in Tensorboard, there is a significant reduction between the `orginial` and `compression` curves in the `aggregated_bits` plots while in the `loss` and `sparse_categorical_accuracy` plot the two curves are pretty similiar.

결론적으로, 원래의 Federated Averaging 알고리즘과 유사한 성능을 달성 할 수 있는 압축 알고리즘을 구현하면서 통신 비용은 대폭 절감했습니다.

In [ ]:
#@test {"skip": true}
%tensorboard --logdir /tmp/logs/scalars/ --port=0

## 연습

사용자 정의 압축 알고리즘을 구현하고 훈련 루프에 적용하기 위해 다음을 수행할 수 있습니다.

1. Implement a new compression algorithm as a subclass of [tff.aggregators.MeanFactory](https://www.tensorflow.org/federated/api_docs/python/tff/aggregators/MeanFactory).
2. Perform training with the compression algorithm to see if it does better than the algorithm above.

잠재적으로 가치 있는 개방형 연구 질문에는 불균일 양자화, 허프만 코딩과 같은 무손실 압축, 이전 훈련 라운드의 정보를 기반으로 한 압축을 조정하는 메커니즘이 포함됩니다.

권장 참고 자료:

- [Expanding the Reach of Federated Learning by Reducing Client Resource Requirements](https://research.google/pubs/pub47774/)
- [Federated Learning: Strategies for Improving Communication Efficiency](https://research.google/pubs/pub45648/)
- [Advanced and Open Problems in Federated Learning](https://arxiv.org/abs/1912.04977)의 *Section 3.5 Communication and Compression*